In [23]:
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/hello/', methods=['GET', 'POST'])
def welcome():
    return "Hello World!"

@app.route('/predict/', methods=['POST'])
def prediction():
    data = request.get_json()
    error = validate_data(data)
    if error:
        print("ERROR:", error)
        return jsonify({"error": error,}), 403
    
    return "Success", 200

def validate_data(data):
    error = []
   
    data_to_dict = data
    
    if data['city'] == '':
        error.append("city field is empty")
        
    if data['age'] == '':
        error.append("Age field is empty")
    
    if data['calls'] == '':
        error.append("calls field is empty")
        
    if data['activeCalls'] == '':
        error.append("activeCalls field is empty")
        
    if data['sms'] == '':
        error.append("SMS field is empty")
        
    if data['data'] == '':
        error.append("Data usage field is empty")
        
    if data['subscription_a'] == '':
        error.append("Subscription A field is empty")
        
    if data['subscription_b'] == '':
        error.append("Subscription B field is empty")
        
    if data['subscription_c'] == '':
        error.append("Subscription C field is empty")
        
    if data['subscription_d'] == '':
        error.append("Subscription D field is empty")
    
    return error
    

if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Apr/2021 16:07:23] "POST /predict/ HTTP/1.1" 403 -


{'name': 'Tom Cruise', 'address': '102, Silicon Valley, USA', 'phoneNumber': '+1-306-698-254', 'city': 'Nivada', 'age': 23, 'calls': 2, 'activeCalls': 23, 'sms': 6, 'data': 1024, 'subscription_a': True, 'subscription_b': False, 'subscription_c': False, 'subscription_d': '', 'plan': 5}
ERROR: ['Subscription D field is empty']


127.0.0.1 - - [10/Apr/2021 16:07:41] "POST /predict/ HTTP/1.1" 403 -


{'name': 'Tom Cruise', 'address': '102, Silicon Valley, USA', 'phoneNumber': '+1-306-698-254', 'city': 'Nivada', 'age': 23, 'calls': 2, 'activeCalls': 23, 'sms': 6, 'data': 1024, 'subscription_a': True, 'subscription_b': False, 'subscription_c': '', 'subscription_d': '', 'plan': 5}
ERROR: ['Subscription C field is empty', 'Subscription D field is empty']


127.0.0.1 - - [10/Apr/2021 16:08:06] "POST /predict/ HTTP/1.1" 200 -


{'name': 'Tom Cruise', 'address': '102, Silicon Valley, USA', 'phoneNumber': '+1-306-698-254', 'city': 'Nivada', 'age': 23, 'calls': 2, 'activeCalls': 23, 'sms': 6, 'data': 1024, 'subscription_a': True, 'subscription_b': False, 'subscription_c': False, 'subscription_d': False, 'plan': 5}


In [50]:
import numpy as np
import pandas as pd

from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
from matplotlib.pyplot import figure
from sklearn import preprocessing

from flask import Flask, jsonify

le = preprocessing.LabelEncoder()

filename = "E:\\Work\\FinalProjectData\\record_second.csv"
names = ['name', 'address', 'phoneNumber', 'region','age', 'callCount', 'activeCallTime', 
 'smsCount', 'dataUsage', 'Subscription_1', 'Subscription_2', 
 'Subscription_3', 'Subscription_4', 'plan']
dataset = pd.read_csv(filename, names=names)

# The below values are scrapped since they do not contribute to the prediction
dataset.pop('name')
dataset.pop('address')
dataset.pop('phoneNumber')

#print(dataset.describe())
#print(dataset.groupby('plan').size())

# To convert the string to number. Here city names are converted to integer.
le = LabelEncoder()
for col in dataset.columns:
    if col == 'region':
        le.fit(dataset[col])
        dataset[col] = le.transform(dataset[col])

array = dataset.values
x = array[:,0:9]
y = array[:,10]

# Splitting the dataset into training and test set.  
from sklearn.model_selection import train_test_split  
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.25, random_state=0)  

from sklearn.ensemble import RandomForestClassifier  
classifier= RandomForestClassifier(n_estimators= 100, criterion="entropy")  
classifier.fit(x_train, y_train)    

def get_data_to_predict(data):
    data.pop('name')
    data.pop('address')
    data.pop('phoneNumber')
    data['region'] = le.transform([data['region']])[0]
    print(data)
    return data

'''
low_limit = 0
upper_limit = 1
for i in range(20):
    y_pred= classifier.predict(x_test[low_limit:upper_limit])
    #print(y_pred)
    if(y_pred != y_test[low_limit:upper_limit]):
        print("upper_limit: ", upper_limit, y_pred)
       
    low_limit = low_limit + 1
    upper_limit = upper_limit + 1
'''

'''
# Read CSV file and import the data as DataFrame
file_actual = "E:\\Work\\FinalProjectData\\record_actual_data.csv"
dataset_actual = pd.read_csv(file_actual, names=names)

# The below values are scrapped since they do not contribute to the prediction
dataset_actual.pop('name')
dataset_actual.pop('address')
dataset_actual.pop('phoneNumber')

#print(dataset.describe())
#print(dataset.groupby('plan').size())
        
# To convert the string to number. Here city names are converted to integer.
for col in dataset_actual.columns:
    if col == 'region':
        dataset_actual[col] = LabelEncoder().fit_transform(dataset_actual[col])

array_actual = dataset_actual.values
x_actual = array_actual[:,0:9]
y_actual = array_actual[:,10]

y_pred= classifier.predict(x_actual)

for i in range(0, len(x_actual)):
    if y_actual[i] != y_pred[i]:
        print("Record: ", i+1, " is not in correct plan. Predicted Plan: ", y_pred[i])

'''


app = Flask(__name__)
prediction_model = get_prediction_model()

@app.route('/hello/', methods=['GET', 'POST'])
def welcome():
    return "Hello World!"

@app.route('/predict/', methods=['POST'])
def prediction():
    data = request.get_json()
    error = validate_data(data)
    if error:
        print("ERROR:", error)
        return jsonify({"error": error,}), 403
    
    data = list(get_data_to_predict(data).values())
    x = data[0:9]
    y = data[10]
    
    print(x)
    print(y)
    
    y_pred = classifier.predict([x])
    print(y_pred[0])
    if y != y_pred[0]:
        return jsonify({"Proposed Plan": str(y_pred[0]),}), 200
    
    return "Success", 200

def validate_data(data):
    error = []
   
    data_to_dict = data
    
    if data['region'] == '':
        error.append("region field is empty")
        
    if data['age'] == '':
        error.append("Age field is empty")
    
    if data['calls'] == '':
        error.append("calls field is empty")
        
    if data['activeCalls'] == '':
        error.append("activeCalls field is empty")
        
    if data['sms'] == '':
        error.append("SMS field is empty")
        
    if data['data'] == '':
        error.append("Data usage field is empty")
        
    if data['subscription_a'] == '':
        error.append("Subscription A field is empty")
        
    if data['subscription_b'] == '':
        error.append("Subscription B field is empty")
        
    if data['subscription_c'] == '':
        error.append("Subscription C field is empty")
        
    if data['subscription_d'] == '':
        error.append("Subscription D field is empty")
    
    return error
    

if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Apr/2021 17:09:48] "POST /predict/ HTTP/1.1" 200 -


{'region': 1, 'age': 45, 'calls': 50, 'activeCalls': 220, 'sms': 60, 'data': 8095, 'subscription_a': 1, 'subscription_b': 1, 'subscription_c': 1, 'subscription_d': 1, 'plan': 1}
[1, 45, 50, 220, 60, 8095, 1, 1, 1]
1
7


127.0.0.1 - - [10/Apr/2021 17:10:09] "POST /predict/ HTTP/1.1" 200 -


{'region': 1, 'age': 45, 'calls': 50, 'activeCalls': 220, 'sms': 60, 'data': 8095, 'subscription_a': 1, 'subscription_b': 1, 'subscription_c': 1, 'subscription_d': 1, 'plan': 1}
[1, 45, 50, 220, 60, 8095, 1, 1, 1]
1
7
